In [1]:
%pip install pymongo


  Using cached pymongo-4.10.1-cp311-cp311-win_amd64.whl.metadata (22 kB)
  Using cached dnspython-2.7.0-py3-none-any.whl.metadata (5.8 kB)
Using cached pymongo-4.10.1-cp311-cp311-win_amd64.whl (876 kB)
Using cached dnspython-2.7.0-py3-none-any.whl (313 kB)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
from pymongo import MongoClient

# Connect to MongoDB (Replace with your Atlas connection string if necessary)
client = MongoClient("mongodb://localhost:27017/")

# Create a new database
db = client["ecommerce"]

# Create collections
customers = db["customers"]
products = db["products"]
orders = db["orders"]
order_items = db["order_items"]

# Insert sample data into customers collection
customers.insert_many([
    {"customer_id": 1, "name": "Alice", "email": "alice@example.com", "address": {"street": "123 Maple St", "city": "Springfield", "state": "IL"}},
    {"customer_id": 2, "name": "Bob", "email": "bob@example.com", "address": {"street": "456 Oak St", "city": "Metropolis", "state": "NY"}}
])

# Insert sample data into products collection
products.insert_many([
    {"product_id": 101, "product_name": "Laptop", "category": "Electronics", "price": 1200},
    {"product_id": 102, "product_name": "Phone", "category": "Electronics", "price": 800}
])

# Insert sample data into orders collection
orders.insert_many([
    {"order_id": 5001, "customer_id": 1, "order_date": "2024-01-15T10:00:00Z", "status": "Delivered"},
    {"order_id": 5002, "customer_id": 2, "order_date": "2024-02-01T14:00:00Z", "status": "Delivered"}
])

# Insert sample data into order_items collection
order_items.insert_many([
    {"order_item_id": 9001, "order_id": 5001, "product_id": 101, "quantity": 2, "price": 1200},
    {"order_item_id": 9002, "order_id": 5002, "product_id": 102, "quantity": 1, "price": 800}
])


InsertManyResult([ObjectId('67683a74d4857a2855d51428'), ObjectId('67683a74d4857a2855d51429')], acknowledged=True)

In [3]:
pipeline = [
    {"$lookup": {
        "from": "order_items", 
        "localField": "order_id", 
        "foreignField": "order_id", 
        "as": "order_details"
    }},
    {"$unwind": "$order_details"},
    {"$group": {
        "_id": "$order_details.product_id", 
        "total_revenue": {"$sum": {"$multiply": ["$order_details.quantity", "$order_details.price"]}}
    }},
    {"$sort": {"total_revenue": -1}}
]

result = list(db.orders.aggregate(pipeline))
print(result)


[{'_id': 101, 'total_revenue': 2400}, {'_id': 102, 'total_revenue': 800}]


In [4]:
#What is the average delivery time for orders?
pipeline = [
    {"$project": {
        "order_date": 1,
        "delivery_date": 1,
        "delivery_time": {"$subtract": ["$delivery_date", "$order_date"]}
    }},
    {"$group": {
        "_id": None,
        "average_delivery_time": {"$avg": "$delivery_time"}
    }}
]

result = list(db.orders.aggregate(pipeline))
print(result)


[{'_id': None, 'average_delivery_time': None}]


In [5]:
#Which states have the highest number of customers?
pipeline = [
    {"$group": {
        "_id": "$address.state", 
        "customer_count": {"$sum": 1}
    }},
    {"$sort": {"customer_count": -1}}
]

result = list(db.customers.aggregate(pipeline))
print(result)


[{'_id': 'NY', 'customer_count': 1}, {'_id': 'IL', 'customer_count': 1}]


In [6]:
#Top 3 most expensive products sold in each order
pipeline = [
    {"$lookup": {
        "from": "order_items", 
        "localField": "order_id", 
        "foreignField": "order_id", 
        "as": "order_details"
    }},
    {"$unwind": "$order_details"},
    {"$sort": {"order_details.price": -1}},
    {"$group": {
        "_id": "$order_id",
        "top_products": {"$push": "$order_details.product_id"},
    }},
    {"$limit": 3}
]

result = list(db.orders.aggregate(pipeline))
print(result)


[{'_id': 5002, 'top_products': [102]}, {'_id': 5001, 'top_products': [101]}]


In [8]:
# optimizing schema design
# Create an index on customer_id for faster queries
db.orders.create_index([("customer_id", 1)])

# Create an index on product_id for faster queries
db.order_items.create_index([("product_id", 1)])


'product_id_1'

In [10]:

# implement transaction
session = client.start_session()

with session.start_transaction():
    try:
        # Insert a new order
        order = {"order_id": 5003, "customer_id": 1, "order_date": "2024-03-01T09:00:00Z", "status": "Pending"}
        db.orders.insert_one(order, session=session)

        # Update product inventory (for example, decrease stock quantity)
        db.products.update_one({"product_id": 101}, {"$inc": {"stock_quantity": -1}}, session=session)
    except Exception as e:
        print(f"Error: {e}")
        session.abort_transaction()

session.end_session()


Error: Transaction numbers are only allowed on a replica set member or mongos, full error: {'ok': 0.0, 'errmsg': 'Transaction numbers are only allowed on a replica set member or mongos', 'code': 20, 'codeName': 'IllegalOperation'}


In [12]:
# applying change stream
pipeline = [{'$match': {'operationType': 'insert'}}]

with db.orders.watch(pipeline=pipeline) as stream:
    for change in stream:
        print(change)


OperationFailure: The $changeStream stage is only supported on replica sets, full error: {'ok': 0.0, 'errmsg': 'The $changeStream stage is only supported on replica sets', 'code': 40573, 'codeName': 'Location40573'}